In [1]:
import numpy as np
import os
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import cv2
import torch
import tensorflow as tf
import time
import pandas as pd
import collections
from collections import Counter
from deepface import DeepFace
from utility.yolov5_output_processing import processing_yolov5
from deep_sort.deep_sort import DeepSort
from utility.utility_functions import write_person_data_toFile
from utility.utility_functions import person_data_update

from deepface_similarity_matching.face_similarity_matching import find_similars

/home/chakradhar/.conda/envs/multimodular/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/chakradhar/.conda/envs/multimodular/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/chakradhar/.conda/envs/multimodular/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/chakradhar/.conda/envs/multimodular/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
from body_pose_recognition.body_features_extraction_train import body_features_extraction
from face_emotion_recognition.face_feature_extraction_pyfeat import face_features_extraction

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

def engagement_plot_hist(y, plot_width=480, plot_height=200, dpi=200):
    x = (0, 1)
    fig = plt.figure(dpi=dpi)
    opacity = 0.4
    bar_width = 0.2

    plt.xlabel('Engagement')
    plt.ylabel('Count')

    plt.xticks(x, ('Pos', 'Neg'), rotation=30)
    
    #plt.gcf().canvas.draw()
    plt.gcf().set_size_inches(1.5, 2)
    bar1 = plt.bar(x, y, bar_width, align='center', alpha=opacity, color='b')

    # Add counts above the two bar graphs
    for rect in bar1:
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width() / 2.0, height, f'{height:.0f}', ha='center', va='bottom')

    #plt.legend()
    plt.tight_layout()    
    fig.canvas.draw()

    # Now we can save it to a numpy array.
    data2 = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    data2 = data2.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    #data2 = cv2.resize(data2, (plot_width, plot_height))
    #cv2.imwrite('test2.png',data2)
    plt.close()
    
    return data2

def engagement_plot_line(y, z, plot_width=480, plot_height=200, dpi=100):
    x = list(range(len(y)))

    fig = plt.figure()
    
    plt.xlabel('Frame')
    plt.ylabel('Count')
    
    #plt.gcf().canvas.draw()
    #plt.gcf().set_size_inches(1.5, 2)
    plt.plot(x, y, color='g', label='engaged')
    #plt.plot(x, z, color='b', label='non-engaged')

    plt.legend()
    plt.tight_layout()    
    fig.canvas.draw()

    # Now we can save it to a numpy array.
    data2 = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    data2 = data2.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    #data2 = cv2.resize(data2, (plot_width, plot_height))
    #cv2.imwrite('test2.png',data2)
    plt.close()
    
    return data2

def engagement_plot(x, y, plot_width=480, plot_height=200, dpi=200):
    
    plot_hist = engagement_plot_hist([x[-1], y[-1]], plot_width, plot_height, dpi)
    plot_line = engagement_plot_line(x, y, plot_width, plot_height, dpi)
    plot = np.ones((plot_hist.shape[0] + plot_line.shape[0], max(plot_line.shape[1], plot_hist.shape[1]), 3), dtype=np.uint8) * 255
    
    plot[:plot_hist.shape[0], :plot_hist.shape[1], :] = plot_hist
    plot[plot_hist.shape[0]:plot_hist.shape[0] + plot_line.shape[0], :plot_line.shape[1], :] = plot_line
    
    return plot

In [32]:
def person_tracking_with_behavior_classification_video(input_video_path, base_path = '../project heavy/runs', 
                                                       sampling_rate = 5, time_period = 10, 
                                                       detections_len_threshold = 5):
    """
    Inputs:
    sampling_rate: total number of frames to take for every 1 sec
    time_period: time in seconds after which averaging is done
    detections_len_threshold: a track should have this minimum number of classifications to be a valid one,
                                It is used while writing to csv.
    
        
    """
    
    # 
    # output folder setup
    #
    if os.listdir(base_path):
        run_num = max((int(runs[3:]) for runs in os.listdir(base_path))) + 1
    else:
        run_num = 1
    output_base_path = os.path.join(base_path, 'run'+str(run_num))
    os.mkdir(output_base_path) 
    video_path = os.path.join(output_base_path, 'result.avi')
    
    #
    # Yolov5 medium
    # 
    model_yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5m')
    
    #
    # DeepSort object
    #
    deep_sort = DeepSort()
    
    #
    # body_behavior_classifier function
    #
    bf_ex, num_bf = body_features_extraction()
    
    #
    # face_emotion_classifier function
    #
    ff_ex, num_ff = face_features_extraction()
    
    #
    # Engagement classification model
    #
    engagement_model = tf.keras.models.load_model('../body_face_dnn_model_250622')

    #
    # Data Collection
    #
    person_data = dict()
    track_id_behavior = dict()
    person_average_behavior = dict()
    
    #
    # Graph plotting
    #
    count_engaged_li = [0]
    count_non_engaged_li = [0]
    
    #
    # video input and setting up the output video
    #
    vid = cv2.VideoCapture(input_video_path)
    codec = cv2.VideoWriter_fourcc(*'XVID')
    vid_fps =int(vid.get(cv2.CAP_PROP_FPS))
    vid_width, vid_height = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)), int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out_width, out_height = 1280, 600
    out = cv2.VideoWriter(video_path, codec, 1, (out_width, out_height))
    
    frames_in_seconds = sampling_rate;
    sampling_rate = int(vid_fps // sampling_rate)

    print("Analyzing Video: {}, \nSampling Rate: {}".format(video_path, sampling_rate))
    print(vid_fps)
    count_frame_classified = 0
    iteration_ = 0
    frame_num_li = []
    
    start_time = time.time()
    while True:
        _, img = vid.read()
        if img is None:
            print('Completed')
            break

        iteration_ += 1
        
        #for sampling of frames
        if iteration_ % sampling_rate != 0:
            continue
            
        count_frame_classified += 1
        frame_num_li.append(iteration_)
        
        img_in = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_rgb = img_in.copy()
        img_write = img.copy()

        results_yolov5 = model_yolov5(img_in, size=640)
        names, scores, converted_boxes, converted_boxes_mobile = processing_yolov5(results_yolov5)

        #
        # Apply the DeepSort and get the tracker object
        #
        deep_sort.deep_sort_module(img, names, scores, converted_boxes)
        
        count_valid_tracks = 0
        for track in deep_sort.tracker.tracks:
            if not track.is_confirmed() or track.time_since_update >1:
                continue
            count_valid_tracks += 1    
        if count_valid_tracks == 0:
            continue
        
        #
        # Extract the body features
        #
        body_id, b_features, img_openpose = bf_ex(img, deep_sort.tracker, converted_boxes_mobile)
        b_features_dict = {}
        for idx, Id in enumerate(body_id):
            b_features_dict[Id] = idx
            
        img = img_openpose
       
        #
        # Extract the face features
        #
        face_id, f_features = ff_ex(img_rgb, deep_sort.tracker)
        f_features_dict = {}
        for idx, Id in enumerate(face_id):
            f_features_dict[Id] = idx
        
        frame_level_person_behavior = {}
        #
        # Frame level Classification
        #
        for track in deep_sort.tracker.tracks:
            if not track.is_confirmed() or track.time_since_update >1:
                continue
            Id = track.track_id
            flag = True
            features = np.zeros((1, num_bf + num_ff))
            
            if Id in body_id and Id in face_id:
                features[0,:num_bf] = b_features[b_features_dict[Id]]
                features[0,num_bf:] = f_features[f_features_dict[Id]]
                
            elif Id in body_id:
                features[0,:num_bf] = b_features[b_features_dict[Id]]
            
            elif Id in face_id:
                features[0,num_bf:] = f_features[f_features_dict[Id]]
            
            else:
                flag = False
            x1, y1, x2, y2 = [int(coord) for coord in track.to_tlbr()]    
            if flag:
                #print(features.shape)
                engagement_level = engagement_model.predict(features, verbose=False)[0]
                #print(engagement_level)
                if track_id_behavior.get(Id, None) is None:
                    track_id_behavior[Id] = list()
                track_id_behavior[Id].append(1 if engagement_level > 0.5 else 0)
                frame_level_behavior = 'pos' if engagement_level > 0.5 else 'neg'
                frame_level_person_behavior[Id] = frame_level_behavior
                # drawing the bbox to the frame and frame level classification
                if frame_level_behavior == 'pos':
                    cv2.rectangle(img, (x1, y1), (x2, y2),(0,255,0), 2)
                    cv2.putText(img, "eng. lev.: " + frame_level_behavior +"- ID: "+str(track.track_id), 
                                (x1, y1-10), 0, 0.75, (255, 255, 255), 2)
                else:
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0,0,255), 2)
                    cv2.putText(img, "eng. lev.: " + frame_level_behavior +"- ID: "+str(track.track_id), 
                                (x1, y1-10), 0, 0.75, (255, 255, 255), 2)
            else:
                # if the classification is undefined
                cv2.rectangle(img, (x1, y1), (x2, y2), (255,0,0), 2)
                cv2.putText(img, "eng. lev.: " + 'undefined' +"- ID: "+str(track.track_id), 
                            (x1, y1-10), 0, 0.75, (255, 255, 255), 2)
                
                
        #
        # Averaging the Classification
        #
        count_engaged, count_non_engaged = 0, 0
        if count_frame_classified == time_period * frames_in_seconds:
            flag_tmp = True
            for id_, data_ in track_id_behavior.items():
                cntr = Counter(data_)
                if cntr.most_common(1)[0][1] == len(person_average_behavior) // 2:
                    person_average_behavior[id_] = 'Not Defined'
                else:
                    if cntr.most_common(1)[0][0] == 1:
                        person_average_behavior[id_] = 'pos'
                        count_engaged += 1
                    else:
                        person_average_behavior[id_] = 'neg'
                        count_non_engaged += 1
            count_engaged_li.append(count_engaged)
            count_non_engaged_li.append(count_non_engaged)  
            #
            # Updating the trackId data in person_data
            #
            person_data_update(person_data, img_write, deep_sort.tracker, iteration_, person_average_behavior)
            
            #
            # resetting the dictionaries and values
            #
            count_frame_classified = 0
            person_average_behavior = {}
            track_id_behavior = {}
            
        #
        # Frame level Classification into CSV
        #
        else:
            person_data_update(person_data, img_write, deep_sort.tracker, iteration_, frame_level_person_behavior)
            
        #
        # plotting section
        #
        
        #
        # combining image frame with plot
        #
        plot = engagement_plot(count_engaged_li, count_non_engaged_li)

        new_frame_width, new_frame_height = img.shape[1] + plot.shape[1], max(img.shape[0], plot.shape[0])
        new_frame = np.ones((new_frame_height, new_frame_width, 3), dtype=np.uint8) * 255
        new_frame[:img.shape[0],:img.shape[1], :] = img
        new_frame[:plot.shape[0], img.shape[1]:new_frame_width, :] = plot

        # resizing of the new_frame as per output video specifications
        new_frame = cv2.resize(new_frame, (out_width, out_height))
        # show the output frame
        out.write(new_frame)

        # duplicating the classified frame so that it stays for some time in
        # the video
        if count_engaged + count_non_engaged > 0:
            for i in range(4):
                out.write(new_frame)


#         if count_frame_classified % 10 == 0:
#             print("\tFrame classified: ", count_frame_classified)

                 
        if cv2.waitKey(1) == ord('q'):
            break
    
    print("Total Time taken: ", time.time() - start_time)
    vid.release()
    out.release()

    #
    # write data to File
    #
    write_person_data_toFile(person_data, output_base_path, frame_num_li, detections_len_threshold)
    print("Video at: ", video_path)
    
    #
    # Merging the Multiple tracks of same person
    #
    #print(find_similars(DeepFace, output_base_path))


In [33]:
#person_tracking_with_behavior_classification_video('../project heavy/data/video/class_video1.mp4')
person_tracking_with_behavior_classification_video(r'../person_7_20s.MOV')

Using cache found in /home/chakradhar/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-6-15 Python-3.9.12 torch-1.9.0+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 


Analyzing Video: ../project heavy/runs/run32/result.avi, 
Sampling Rate: 5
25
Completed
Total Time taken:  85.95354127883911
Images and Data written at ../project heavy/runs/run32
Video at:  ../project heavy/runs/run32/result.avi


## Face Similarity to Find Duplicates using DeepFace 

In [8]:
print(find_similars(DeepFace, '../project heavy/runs/run6'))

vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /home/chakradhar/.deepface/weights/vgg_face_weights.h5
100%|████████████████████████████████████████| 580M/580M [01:36<00:00, 6.03MB/s]


1/1 [==============================] - 0s 9ms/step
Representations stored in  ../project heavy/runs/run6/image / representations_vgg_face.pkl  file. Please delete this file when you add new identities in your database.
1/1 [==============================] - 0s 10ms/step
find function lasts  99.29092693328857  seconds
[['7', '1', '6', '4', '3'], ['8'], ['2', '5']]


In [14]:
from deepface import DeepFace

backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface', 'mediapipe']

#face verification
obj = DeepFace.verify(img1_path = "../project heavy/runs/run6/image/person_7.jpg", img2_path = "../project heavy/runs/run6/image/person_6.jpg", detector_backend = backends[4])

1/1 [==============================] - 0s 10ms/step


In [15]:
obj

{'verified': True,
 'distance': 0.3115096947027909,
 'threshold': 0.4,
 'model': 'VGG-Face',
 'detector_backend': 'retinaface',
 'similarity_metric': 'cosine'}